In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,mean_squared_error

In [82]:
df = pd.read_csv("data.csv")

In [83]:
df

,mill_date,mill_shift,machine_code,loss_code,loss_name,stop_begin_time_start,stop_begin_time_end,stop_duration
0,01-01-2021,1,VMC-B41-01,903,Work Center Cleaning,01-01-2021 08:00,01-01-2021 08:00,00:00:00
1,01-01-2021,1,VMC-B41-02,903,Work Center Cleaning,01-01-2021 08:00,01-01-2021 08:00,00:00:00
2,01-01-2021,1,VMC-B41-02,903,Work Center Cleaning,01-01-2021 08:02,01-01-2021 08:08,00:05:48
3,01-01-2021,1,VMC-B41-02,405,Tool Setting,01-01-2021 08:08,01-01-2021 08:14,00:05:58
4,01-01-2021,1,VMC-B41-02,405,Tool Setting,01-01-2021 08:20,01-01-2021 08:34,00:13:55
...,...,...,...,...,...,...,...,...
22337,31-01-2021,3,BND-B11-02,1008,Shift end going early,01-02-2021 01:00,01-02-2021 01:00,00:00:00
22338,31-01-2021,3,BND-B11-03,405,Tool Setting,01-02-2021 01:00,01-02-2021 01:00,00:00:00
22339,31-01-2021,3,BND-B11-04,406,Material Movement,01-02-2021 01:00,01-02-2021 01:00,00:00:00
22340,31-01-2021,3,BND-B11-05,406,Material Movement,01-02-2021 01:00,01-02-2021 01:00,00:00:00


In [43]:
df.dtypes

mill_date                object
mill_shift                int64
machine_code             object
loss_code                 int64
loss_name                object
stop_begin_time_start    object
stop_begin_time_end      object
stop_duration            object
dtype: object

In [87]:
len(df["machine_code"].value_counts())

18

In [44]:
df["stop_duration"].value_counts()

00:00:00    1048
00:00:05     427
00:00:04     417
00:00:07     404
00:00:06     401
            ... 
00:17:51       1
00:28:09       1
00:37:24       1
00:15:08       1
00:36:19       1
Name: stop_duration, Length: 1816, dtype: int64

In [45]:
def convert_to_seconds(value):
    hours, minutes, seconds = value.split(':')
    seconds = int(hours) * 60 + int(minutes) * 60 + int(seconds)
    return seconds

In [46]:
df['Stop_Duration_Time(sec)'] = df['stop_duration'].apply(convert_to_seconds)

In [47]:
df

,mill_date,mill_shift,machine_code,loss_code,loss_name,stop_begin_time_start,stop_begin_time_end,stop_duration,Stop_Duration_Time(sec)
0,01-01-2021,1,VMC-B41-01,903,Work Center Cleaning,01-01-2021 08:00,01-01-2021 08:00,00:00:00,0
1,01-01-2021,1,VMC-B41-02,903,Work Center Cleaning,01-01-2021 08:00,01-01-2021 08:00,00:00:00,0
2,01-01-2021,1,VMC-B41-02,903,Work Center Cleaning,01-01-2021 08:02,01-01-2021 08:08,00:05:48,348
3,01-01-2021,1,VMC-B41-02,405,Tool Setting,01-01-2021 08:08,01-01-2021 08:14,00:05:58,358
4,01-01-2021,1,VMC-B41-02,405,Tool Setting,01-01-2021 08:20,01-01-2021 08:34,00:13:55,835
...,...,...,...,...,...,...,...,...,...
22337,31-01-2021,3,BND-B11-02,1008,Shift end going early,01-02-2021 01:00,01-02-2021 01:00,00:00:00,0
22338,31-01-2021,3,BND-B11-03,405,Tool Setting,01-02-2021 01:00,01-02-2021 01:00,00:00:00,0
22339,31-01-2021,3,BND-B11-04,406,Material Movement,01-02-2021 01:00,01-02-2021 01:00,00:00:00,0
22340,31-01-2021,3,BND-B11-05,406,Material Movement,01-02-2021 01:00,01-02-2021 01:00,00:00:00,0


In [48]:
df.groupby(["mill_date","mill_shift","machine_code"]).sum()

loss_code  Stop_Duration_Time(sec)
mill_date  mill_shift machine_code                                    
01-01-2021 1          BND-B11-01        11359                    16461
                      BND-B11-02            1                        0
                      BND-B11-03         5788                     8897
                      BND-B11-04         1009                      713
                      BND-B11-05         7531                    17497
...                                       ...                      ...
31-01-2021 3          VMC-B41-01          903                        0
                      VMC-B41-02         1904                        0
                      VMC-B41-03         1806                        0
                      VMC-B41-04          903                        0
                      VTL-B41-01          402                        0

[1288 rows x 2 columns]

In [49]:
df.columns

Index(['mill_date', 'mill_shift', 'machine_code', 'loss_code', 'loss_name',
       'stop_begin_time_start', 'stop_begin_time_end', 'stop_duration',
       'Stop_Duration_Time(sec)'],
      dtype='object')

In [50]:
df.groupby(["mill_date","loss_name","machine_code"]).sum().head(50)

mill_shift  \
mill_date  loss_name                             machine_code               
01-01-2021 Air Pressure Low                      VMC-B41-04             1   
           Coolent Filling                       CNT-B41-01             1   
                                                 VMC-B41-03             2   
                                                 VMC-B41-04             1   
           Drawing Clarification                 BND-B11-05             3   
           First off Insp.                       VMC-B41-03             1   
           House Keeping                         BND-B11-01             1   
                                                 CNT-B41-01             1   
                                                 CNT-B41-03             1   
           Insert Change                         CNT-B41-02             5   
                                                 VMC-B41-02             2   
           Lunch & Dinner Time                   BND-B11-01             1   
                                                 BND-B11-05             1   
                                                 CNT-B41-01             3   
                                                 CNT-B41-02             1   
                                                 CNT-B41-03             3   
                                                 VMC-B41-02             3   
                                                 VMC-B41-03             3   
                                                 VMC-B41-04             1   
           Material Movement                     BND-B11-01            12   
                                                 BND-B11-03             6   
                                                 BND-B11-05             3   
           No Man Power                          BND-B11-06             6   
                                                 CNT-B41-02             7   
                                                 CNT-B41-04             5   
                                                 VMC-B41-03             4   
                                                 VMC-B41-04             2   
           No Plan                               CNT-B41-03             1   
                                                 VMC-B41-02             2   
           Other Reason                          VMC-B41-04             2   
           Other Scheduled Meeting by Management BND-B11-01             1   
                                                 BND-B11-03             1   
                                                 BND-B11-05             1   
                                                 CNT-B41-01             2   
                                                 CNT-B41-02             2   
                                                 CNT-B41-03             2   
                                                 VMC-B41-02             1   
                                                 VMC-B41-03             2   
                                                 VMC-B41-04             1   
           Other Stop-Unkown                     BND-B11-01            73   
                                                 BND-B11-02             1   
                                                 BND-B11-03            35   
                                                 BND-B11-04             1   
                                                 BND-B11-05            80   
                                                 BND-B11-06             1   
                                                 CNT-B41-01            51   
                                                 CNT-B41-02            76   
                                                 CNT-B41-03            49   
                                                 CNT-B41-04             4   
                                                 VMC-B41-02            76   

                                                          

In [51]:
df2=df.groupby(["mill_date","loss_name","loss_code","machine_code"])['Stop_Duration_Time(sec)'].sum()

In [52]:
df2=pd.DataFrame(df2)

In [53]:
df2.head(60)

Stop_Duration_Time(sec)
mill_date  loss_name                             loss_code machine_code                         
01-01-2021 Air Pressure Low                      201       VMC-B41-04                        171
           Coolent Filling                       901       CNT-B41-01                        587
                                                           VMC-B41-03                        327
                                                           VMC-B41-04                        595
           Drawing Clarification                 403       BND-B11-05                        984
           First off Insp.                       1901      VMC-B41-03                        153
           House Keeping                         1904      BND-B11-01                        383
                                                           CNT-B41-01                        574
                                                           CNT-B41-03                        519
           Insert Change                         407       CNT-B41-02                        873
                                                           VMC-B41-02                        280
           Lunch & Dinner Time                   1002      BND-B11-01                       2477
                                                           BND-B11-05                       2572
                                                           CNT-B41-01                       4119
                                                           CNT-B41-02                       2133
                                                           CNT-B41-03                       2273
                                                           VMC-B41-02                       4165
                                                           VMC-B41-03                       1938
                                                           VMC-B41-04                       2073
           Material Movement                     406       BND-B11-01                       2835
                                                           BND-B11-03                       3889
                                                           BND-B11-05                       4195
           No Man Power                          1902      BND-B11-06                       2892
                                                           CNT-B41-02                       3397
                                                           CNT-B41-04                        106
                                                           VMC-B41-03                        220
                                                           VMC-B41-04                       1307
           No Plan                               1903      CNT-B41-03                        430
                                                           VMC-B41-02                       1881
           Other Reason                          1201      VMC-B41-04                       2261
           Other Scheduled Meeting by Management 1303      BND-B11-01                       1830
                                                           BND-B11-03                       1662
                                                           BND-B11-05                       1896
                                                           CNT-B41-01                       1804
                                                           CNT-B41-02                       1425
                                                           CNT-B41-03                       1517
                                                           VMC-B41-02                       1312
                                                           VMC-B41-03                       1586
                                                           VMC-B41-04                       1235
           Other Stop-Unkown                     1         BND-B11-01                       5192

In [54]:
df.dtypes

mill_date                  object
mill_shift                  int64
machine_code               object
loss_code                   int64
loss_name                  object
stop_begin_time_start      object
stop_begin_time_end        object
stop_duration              object
Stop_Duration_Time(sec)     int64
dtype: object

In [55]:
def clean_mill_date(df):
    converted_list_1 = []
    for i in df["mill_date"]:
        converted_list_1.append(i.replace('-', ''))
    df["mill_date"]=pd.Series(converted_list_1)

In [56]:
clean_mill_date(df)

In [57]:
df.head()

,mill_date,mill_shift,machine_code,loss_code,loss_name,stop_begin_time_start,stop_begin_time_end,stop_duration,Stop_Duration_Time(sec)
0,01012021,1,VMC-B41-01,903,Work Center Cleaning,01-01-2021 08:00,01-01-2021 08:00,00:00:00,0
1,01012021,1,VMC-B41-02,903,Work Center Cleaning,01-01-2021 08:00,01-01-2021 08:00,00:00:00,0
2,01012021,1,VMC-B41-02,903,Work Center Cleaning,01-01-2021 08:02,01-01-2021 08:08,00:05:48,348
3,01012021,1,VMC-B41-02,405,Tool Setting,01-01-2021 08:08,01-01-2021 08:14,00:05:58,358
4,01012021,1,VMC-B41-02,405,Tool Setting,01-01-2021 08:20,01-01-2021 08:34,00:13:55,835


In [58]:
df['mill_date']=df.mill_date.str.split(" ",expand=True,)

In [59]:
df['mill_date']

0        01012021
1        01012021
2        01012021
3        01012021
4        01012021
           ...   
22337    31012021
22338    31012021
22339    31012021
22340    31012021
22341    31012021
Name: mill_date, Length: 22342, dtype: object

In [60]:
df['mill_date']=df["mill_date"].astype("int64")

In [61]:
df.dtypes

mill_date                   int64
mill_shift                  int64
machine_code               object
loss_code                   int64
loss_name                  object
stop_begin_time_start      object
stop_begin_time_end        object
stop_duration              object
Stop_Duration_Time(sec)     int64
dtype: object

In [62]:
def clean_machine_code(df):
    converted_list_1 = []
    for i in df["machine_code"]:
        converted_list_1.append(i.replace('-', ''))
    df["machine_code"]=pd.Series(converted_list_1)
clean_machine_code(df)

In [63]:
len(df["machine_code"].value_counts())

18

In [64]:
len(df["loss_name"].unique())

34

In [65]:
df = df.drop(["stop_begin_time_start","stop_begin_time_end","stop_duration"],axis = 1)

In [66]:
df

,mill_date,mill_shift,machine_code,loss_code,loss_name,Stop_Duration_Time(sec)
0,1012021,1,VMCB4101,903,Work Center Cleaning,0
1,1012021,1,VMCB4102,903,Work Center Cleaning,0
2,1012021,1,VMCB4102,903,Work Center Cleaning,348
3,1012021,1,VMCB4102,405,Tool Setting,358
4,1012021,1,VMCB4102,405,Tool Setting,835
...,...,...,...,...,...,...
22337,31012021,3,BNDB1102,1008,Shift end going early,0
22338,31012021,3,BNDB1103,405,Tool Setting,0
22339,31012021,3,BNDB1104,406,Material Movement,0
22340,31012021,3,BNDB1105,406,Material Movement,0


In [67]:
machine_code = pd.get_dummies(df["machine_code"])
loss_name = pd.get_dummies(df["loss_name"])

In [68]:
df_final = pd.concat([df,machine_code,loss_name], axis=1)

In [69]:
df_final = df_final.drop(["machine_code","loss_name"],axis = 1)

In [70]:
df_final

,mill_date,mill_shift,loss_code,Stop_Duration_Time(sec),BNDB1101,BNDB1102,BNDB1103,BNDB1104,BNDB1105,BNDB1106,...,Scrap Cleaning,Shift end going early,Tea time,Tool Setting,Trail,Training,Waiting For QC,Want of Tool,Work Center Cleaning,Wrong tool Issue - Tool Change
0,1012021,1,903,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1012021,1,903,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1012021,1,903,348,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1012021,1,405,358,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1012021,1,405,835,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22337,31012021,3,1008,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
22338,31012021,3,405,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
22339,31012021,3,406,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
22340,31012021,3,406,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
df_final.to_csv('final value.csv',index=False)

In [72]:
X = df_final.drop("Stop_Duration_Time(sec)",axis = 1)
Y = df_final["Stop_Duration_Time(sec)"]

In [73]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 1,test_size = 0.25)

In [74]:
model = LinearRegression()

In [75]:
model.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [76]:
import pickle
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [77]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

0.5184774245288652


In [80]:
Y_pred = model.predict(X_test)
mean_squared_error(Y_pred,Y_test)

88120.60610327651

In [79]:
value = [[20210101,1,1002,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
predictions = model.predict(value)
print(predictions[0])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 55 is different from 40)